In [3]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime


In [43]:
url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Zn_cash'

response = requests.get(url=url)


In [54]:
df = pd.read_html(response.text)[0][:30]
data = df.iloc[:, :2]
data = data.query("date != 'date'")

data['date'] = pd.to_datetime(data['date'])
data.iloc[:,1] = pd.to_numeric(data.iloc[:,1])

data.rename(columns={data.columns[1]:'Zinc'},inplace=True)
data

/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_91189/1550713700.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text)[0][:30]


,date,Zinc
0,2023-10-31,2432.5
1,2023-10-30,2463.5
2,2023-10-27,2445.5
3,2023-10-26,2432.0
4,2023-10-25,2458.0
5,2023-10-24,2416.5
6,2023-10-23,2404.0
7,2023-10-20,2415.5
8,2023-10-19,2408.0
9,2023-10-18,2431.0


In [60]:
def new_westmetall():

    def get_data(metall, col_name):
        url = f'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash'
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data['date'] = pd.to_datetime(data['date'])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]:col_name},inplace=True)
        

        return data

    al = get_data(metall='Al', col_name='aluminium')
    cu = get_data(metall='Cu', col_name='copper')
    pb = get_data(metall='Pb', col_name='lead')
    ni = get_data(metall='Ni', col_name='nickel')
    zn = get_data(metall='Zn', col_name='zink')

    result = pd.merge(al, cu, on='date', how='left').merge(pb, on='date', how='left').merge(
        ni, on='date', how='left').merge(zn, on='date', how='left')
    
    old_data = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)
    
    final_data = pd.concat([old_data, result], axis=0)
    
    final_data.drop_duplicates(subset='date', inplace=True)
    
    final_data.sort_values(by='date', inplace=True)
    
    with pd.ExcelWriter(
        "../parser_beta/data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final_data.to_excel(writer, sheet_name='LME_westmetall')
    
    return final_data

In [78]:
new_westmetall()


/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_91189/1816933899.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text)[0][:30]
/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_91189/1816933899.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_91189/1816933899.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,date,aluminium,copper,lead,nickel,zink
0,2023-10-31,2235.0,8038.0,2110.5,17825.0,2432.5
1,2023-10-30,2252.0,8110.0,2154.5,18180.0,2463.5
2,2023-10-27,2170.5,7963.5,2141.0,18000.0,2445.5
3,2023-10-26,2202.5,7939.0,2131.0,17900.0,2432.0
4,2023-10-25,2188.5,7946.0,2134.0,18150.0,2458.0
5,2023-10-24,2146.5,7900.5,2157.0,17800.0,2416.5
6,2023-10-23,2144.5,7824.5,2171.0,18135.0,2404.0
7,2023-10-20,2152.0,7876.0,2129.0,18350.0,2415.5
8,2023-10-19,2150.5,7938.0,2117.0,18285.0,2408.0
9,2023-10-18,2176.5,7947.5,2132.0,18470.0,2431.0


In [79]:
old_data = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)
old_data

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
456,2023-10-25,2188.5,7946.0,2134.0,18150,2458.0
457,2023-10-26,2202.5,7939.0,2131.0,17900,2432.0
458,2023-10-27,2170.5,7963.5,2141.0,18000,2445.5
459,2023-10-30,2252.0,8110.0,2154.5,18180,2463.5


In [80]:
final_test = pd.concat([old_data, test], axis=0)
final_test

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
24,2023-09-27,2196.0,8017.0,2185.5,18700.0,2464.0
25,2023-09-26,2195.0,8066.0,2222.5,18675.0,2508.0
26,2023-09-25,2182.5,8107.0,2240.0,18950.0,2504.0
27,2023-09-22,2199.5,8188.0,2216.0,19050.0,2532.5


In [81]:
x = final_test.drop_duplicates(subset='date')

In [83]:
x.sort_values(by='date', inplace=True)


/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_91189/3505864503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.sort_values(by='date', inplace=True)


In [84]:
x

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
456,2023-10-25,2188.5,7946.0,2134.0,18150,2458.0
457,2023-10-26,2202.5,7939.0,2131.0,17900,2432.0
458,2023-10-27,2170.5,7963.5,2141.0,18000,2445.5
459,2023-10-30,2252.0,8110.0,2154.5,18180,2463.5
